In [3]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
#usa una GPU se disponibile
device = torch.device('cpu')
print(f"Using device: {device}") # 'cuda' if torch.cuda.is_available() else
try:
  import google.colab
  COLAB = True
  print("Note: using colab")
except:
  print("Note: not using colab")

drive.mount('/content/drive')

#uploaded = files.upload()

Using device: cpu
Note: using colab
Mounted at /content/drive


In [ ]:
class Net(nn.Module):
    def __init__(self, in_count, out_count, hidden_layers, neurons_per_layer, dropout_rates):
        super(Net, self).__init__()
        self.hidden_layers = hidden_layers
        self.neurons_per_layer = neurons_per_layer

        # Define input layer
        self.input_layer = nn.Linear(in_count, neurons_per_layer[0])

        # Define hidden layers
        self.hidden = nn.ModuleList()
        for i in range(self.hidden_layers):
            self.hidden.append(nn.Linear(neurons_per_layer[i], neurons_per_layer[i+1]))

        # Define output layer
        self.output_layer = nn.Linear(neurons_per_layer[-1], out_count)

        # Define dropout layers with different dropout rates
        self.dropouts = nn.ModuleList([nn.Dropout(p=dropout_rates[i]) for i in range(len(dropout_rates))])

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for i in range(self.hidden_layers):
            x = F.relu(self.hidden[i](x))
            x = self.dropouts[i](x)  # Apply dropout with rate dropout_rates[i]
        x = self.output_layer(x)
        return x

In [ ]:
def objective(trial, dataset, labels, device, batch_size, num_epochs):
  dropout_rates = [trial.suggest_float(f'dropout_rate_{i}', 0.1, 0.5) for i in range(5)] #suggerisci dropout rates
  hidden_layers = trial.suggest_int('hidden_layers', 1, 5) #suggerisci numero di hidden layers
  neurons_per_layer = [trial.suggest_int(f'neurons_per_layer_{i}', 16, 1024) for i in range(hidden_layers+1)] #suggerisci il  numero di neuroni per layer
  learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1) #suggerisci learning rate

  #Inizializza il modello con gli iperparametri suggeriti
  model = Net(in_count=dataset.shape[1], out_count=1, hidden_layers=hidden_layers, neurons_per_layer=neurons_per_layer, dropout_rates=dropout_rates)

  #definizione datasets e dataloaders
  X_train, X_val, y_train, y_val = train_test_split(dataset, labels, test_size = 0.2, random_state=42)
  X_train = torch.Tensor(X_train.values).float()
  y_train = torch.Tensor(y_train.values).float()
  X_val = torch.Tensor(X_val.values).float().to(device)
  y_val = torch.Tensor(y_val.values).float().to(device)
  batch_size = batch_size
  dataset_train = TensorDataset(X_train, y_train.unsqueeze(1))
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  dataset_val = TensorDataset(X_val, y_val.unsqueeze(1))
  dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

  #loss function ed ottimizzatore
  criterion = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters())

  #addestramento del modello
  for epoch in range(num_epochs):
    model.train()
    for inputs, labels in dataloader_train:
      #Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, labels)

      #Backward pass e ottimizzazione
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    #Validation loop
    model.eval()
    with torch.no_grad():
      for inputs, labels in dataloader_val:
        outputs = model(inputs)
        val_loss = criterion(outputs, labels)

  #Computa e ritorna loss sul validation set
  return val_loss.item()

In [ ]:
#dataset e parametri
batch_size = 5094
num_epochs = 20
n_trials = 50

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/training_set.csv")
X_df = df.drop(columns=['SWP'], axis=1)
y_df = df['SWP']
scaler = StandardScaler()
features = df.drop(['Unnamed: 0', 'ID', 'ID_Pixel', 'ID_Albero', 'UTM_lon', 'UTM_lat','NDVI', 'SWP'], axis=1).columns.to_list()
df2 = df[features]
X_df_std = pd.DataFrame(scaler.fit_transform(df2[features]),
                                  columns=features,
                                  index=df2.index)
joblib.dump(scaler, '/content/drive/My Drive/Colab Notebooks/scaler.pkl')

study = optuna.create_study(direction='minimize')

with tqdm(total=n_trials, desc="Optimizing") as pbar:
  def callback(study, trial):
    pbar.update(1)
  study.optimize(lambda trial: objective(trial, X_df_std, y_df, device, batch_size, num_epochs),
                 n_trials=n_trials,
                 callbacks=[callback])

best_params = study.best_params
print("Best params:", best_params)

[I 2024-03-13 10:12:34,306] A new study created in memory with name: no-name-3bbf4439-a2e0-447c-bf16-132b5eaab2a8
Optimizing:  84%|████████▍ | 42/50 [6:21:15<1:55:19, 864.96s/it][W 2024-03-13 16:34:43,703] Trial 42 failed with parameters: {'dropout_rate_0': 0.23005897449045326, 'dropout_rate_1': 0.33370670375205885, 'dropout_rate_2': 0.3609546147597148, 'dropout_rate_3': 0.28895180925358355, 'dropout_rate_4': 0.4769393097312337, 'hidden_layers': 5, 'neurons_per_layer_0': 147, 'neurons_per_layer_1': 732, 'neurons_per_layer_2': 808, 'neurons_per_layer_3': 926, 'neurons_per_layer_4': 568, 'neurons_per_layer_5': 827, 'learning_rate': 0.022683527460287095} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-27-6584d0964489>", line 22, in <lambda>
    study.optimize(lambda trial: objective(trial, X_df_

KeyboardInterrupt: 